<a href="https://colab.research.google.com/github/Pri3scilla/FLiT-Data-Science-Mentorship-Projects/blob/main/market_basket_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#load the dataset into pandas dataframe
df=pd.read_csv('/content/Market Basket Analysis - Groceries_dataset (1).csv')

In [ ]:
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [ ]:
df.tail()

,Member_number,Date,itemDescription
38760,4471,08-10-2014,sliced cheese
38761,2022,23-02-2014,candy
38762,1097,16-04-2014,cake bar
38763,1510,03-12-2014,fruit/vegetable juice
38764,1521,26-12-2014,cat food


In [ ]:
df.sample(5)

,Member_number,Date,itemDescription
5885,4607,15-12-2015,ice cream
166,1514,24-08-2015,whole milk
19008,2730,10-11-2015,frozen potato products
11354,1618,13-07-2014,bottled beer
24298,4499,06-05-2015,shopping bags


In [ ]:
df.isnull().sum() #isna

Member_number      0
Date               0
itemDescription    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [ ]:
#convert the date datatype to date
df['Date']=datetime(df[]'Date'])

SyntaxError: ignored